In [1]:
import math
import requests
from bs4 import BeautifulSoup
import json
import time
import random
import numpy as np
from discord_webhook import DiscordWebhook

%load_ext autoreload
%autoreload 2


from six.moves import cPickle as pickle
import six

with open("C:\\Users\\micha\\OneDrive - České vysoké učení technické v Praze\\Dokumenty\\PycharmProjects\\CZC scam bot\\man_vs_woman_suffixes", 'rb') as f:
    mrdka = pickle.load(f)

def _get_matching_suffix(name, suffixes):
    # it is important(!) to try suffixes from longest to shortest
    for suffix_length in six.moves.xrange(len(name), 0, -1):
        suffix = name[-suffix_length:]
        if suffix in suffixes:
            return (suffix, suffixes[suffix])
    return ('', suffixes.get(''))

def sex(name):
    name = six.text_type(name).lower()
    return _get_matching_suffix(name, mrdka)[1] or 'm'

In [2]:

def getNames(url):
    response = session.get('https://www.utvs.cvut.cz/'+url)

    soup = BeautifulSoup(response.content, "html.parser")
    bar = soup.find("table", {"class": "sp-ptbl darkrow"})
    if bar is None:
        return 'Nikdo lol'
    names = bar.find_all("tr", {})[1:]

    res = [name.text[1:-1].replace('\n', ' ') for name in names]
    return res

with requests.session() as session:
    response = session.get('https://www.utvs.cvut.cz/vyuka/seznam-studentu-2?#lerr')

    login = {
        "authtype": "cvut",
        "sw_username": "lukesmi1",
        "sw_password": "Haroldomus321",
        "submit": ""
    }

    r_post = session.post('https://www.utvs.cvut.cz/prihlaseni', data=login)

    response = session.get('https://www.utvs.cvut.cz/vyuka/povinna-volitelna?s%5B%5D=1&s%5B%5D=2&s%5B%5D=33&s%5B%5D=3&s%5B%5D=4&s%5B%5D=125&s%5B%5D=118&s%5B%5D=5&s%5B%5D=124&s%5B%5D=32&s%5B%5D=115&s%5B%5D=121&s%5B%5D=107&s%5B%5D=7&s%5B%5D=9&s%5B%5D=10&s%5B%5D=109&s%5B%5D=11&s%5B%5D=106&s%5B%5D=120&s%5B%5D=108&s%5B%5D=12&s%5B%5D=119&s%5B%5D=128&s%5B%5D=15&s%5B%5D=112&s%5B%5D=16&s%5B%5D=17&s%5B%5D=18&s%5B%5D=19&s%5B%5D=123&s%5B%5D=29&s%5B%5D=105&s%5B%5D=100&s%5B%5D=20&s%5B%5D=34&s%5B%5D=127&s%5B%5D=114&s%5B%5D=21&s%5B%5D=31&s%5B%5D=22&s%5B%5D=24&s%5B%5D=25&s%5B%5D=26&s%5B%5D=129&s%5B%5D=27&s%5B%5D=28&d=0&t=420%3B1380')

    soup = BeautifulSoup(response.content, "html.parser")
    bar = soup.find("table", {"class": "expandable darkrowexp"})

In [3]:
    builder = ''
    data = []
    for line in bar.contents:
        if line != '\n' and line.name == 'tr':
            if len(line.attrs) == 0:
                sport = line.contents[3].text
                day = line.contents[5].text
                time = line.contents[7].text
                place = line.contents[11].text
                teacher = line.contents[13].text

                # print(line.contents[3].text)
            elif line.attrs['class'][0] == 'expansion':
                names = getNames(line.contents[1].contents[12].attrs['href'])
                m = 0
                f = 0
                for name in names:
                    if sex(name) == 'm':
                        m += 1
                    else:
                        f += 1
                ratio = f/(m+f)
                print(sport, day, time, place, teacher, ratio, names)
                builder += f'{sport} {day} {time} {place} {teacher} {ratio} {names}\n'
                data.append([sport, day, time, place, teacher, ratio, names])

    with open('data.txt', 'w') as f:
        f.write(builder)

Tenis Po 08:00 - 09:00 Areál ČVUT Kotlářka Pavel Mátl 0.4 ['Annamária Miheličová', 'Karin Molinková', 'Marek Holík', 'Tommaso Gargiani', 'Martin Indrych', 'Josef Hynek', 'Matěj Čipera', 'Petr Tlamsa', 'Karolína Lišková', 'Lucie Brilová', 'Jan Doskočil', 'Elizaveta Kalmykova', 'Michal Žáček', 'Václav Pejša', 'Anna Špačková']
Kondiční posilování Po 08:00 - 09:00 Strahov - posilovna (blok 2) Čestmír Bulíř 0.32 ['Klára Škrabánková', 'Barbora Kapková', 'Adéla Tichá', 'Aaron Schick', 'Adam Beneš', 'Lukáš Sazima', 'Simona Velichová', 'Vít Maňas', 'Daniel Dynebýr', 'Štěpánka Šípová', 'Richard Kropáček', 'Ondřej Polanecký', 'Jan Koča', 'Jiří Javora', 'Milan Jiříček', 'Martin Kröner', 'Jan Panenka', 'Radek Dvořák', 'Ludvík Prokopec', 'Denisa Frnková', 'Eliška Dudáková', 'Tereza Švábiková', 'Jakub Profota', 'František Dvořák', 'Denis Krajkovič']
Kondiční posilování Po 08:00 - 09:00 Posilovna FEL - Karlovo nám. Daniel Hynek 0.15384615384615385 ['Vratislav Ležal', 'Lukáš Váňa', 'Marian Krotil', 'Da

In [4]:
data.sort(key=lambda x: x[5])
for entry in data:
    print(entry)

['Tenis', 'Po', '12:00 - 13:00', 'Areál ČVUT Kotlářka', 'Martin Wágner', 0.0, ['Filip Kresl', 'Richard Josef Kittrell', 'Daniel Koc', 'Petr Bystroň']]
['Lukostřelba', 'Po', '12:00 - 13:00', 'Lukostřelba KN', 'Martin Kříž', 0.0, ['Adam Blažek']]
['Volejbal', 'Po', '12:00 - 13:30', 'Zelená tělocvična - Juliska', 'Petr Musil', 0.0, ['Vojtěch Tóth', 'Filip Dubják']]
['Tenis', 'Po', '13:00 - 14:00', 'Areál ČVUT Kotlářka', 'Martin Wágner', 0.0, ['Vojtěch Douda', 'Marek Hoffmann', 'Lukáš Hüttner', 'Petr Kuchařík', 'Filip Částka', 'Nikita Kirilovski']]
['Basket', 'Po', '13:30 - 15:00', 'Sokol Vyšehrad', 'Antonín Ludvík', 0.0, ['Vojtěch Hejlek', ' Bauš', 'Kristián Rückl', 'Kryštof Pytelka', 'Petr Přibyl']]
['Kondiční posilování', 'Po', '14:00 - 15:00', 'Posilovna FEL - Karlovo nám.', 'Klára Minaříková', 0.0, 'Nikdo lol']
['Lukostřelba', 'Po', '14:00 - 15:00', 'Lukostřelba - kolej ČVUT Podolí', 'Jiří Horešovský', 0.0, ['Jiří Beneš', 'Václav Znamenáček', ' Fučík']]
['Futsal', 'Po', '15:00 - 16:30